In [2]:
from keras.applications import MobileNet

In [4]:
# MobileNet was designed to work on 224 x 224 pixel input images sizes
img_rows, img_cols = 224, 224 

In [5]:
# Re-loads the MobileNet model without the top or FC layers
MobileNet = MobileNet(weights='imagenet', 
                     include_top = False,
                     input_shape = (img_rows, img_cols, 3))


17235968/17225924 [==============================] - 141s 8us/step


In [6]:
MobileNet.layers

In [7]:
# Layers are set to trainable as True by default
for layer in MobileNet.layers:
    layer.trainable = False

In [8]:
# Let's print our layers 
for (i,layer) in enumerate(MobileNet.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

('0 InputLayer', False)
('1 ZeroPadding2D', False)
('2 Conv2D', False)
('3 BatchNormalization', False)
('4 ReLU', False)
('5 DepthwiseConv2D', False)
('6 BatchNormalization', False)
('7 ReLU', False)
('8 Conv2D', False)
('9 BatchNormalization', False)
('10 ReLU', False)
('11 ZeroPadding2D', False)
('12 DepthwiseConv2D', False)
('13 BatchNormalization', False)
('14 ReLU', False)
('15 Conv2D', False)
('16 BatchNormalization', False)
('17 ReLU', False)
('18 DepthwiseConv2D', False)
('19 BatchNormalization', False)
('20 ReLU', False)
('21 Conv2D', False)
('22 BatchNormalization', False)
('23 ReLU', False)
('24 ZeroPadding2D', False)
('25 DepthwiseConv2D', False)
('26 BatchNormalization', False)
('27 ReLU', False)
('28 Conv2D', False)
('29 BatchNormalization', False)
('30 ReLU', False)
('31 DepthwiseConv2D', False)
('32 BatchNormalization', False)
('33 ReLU', False)
('34 Conv2D', False)
('35 BatchNormalization', False)
('36 ReLU', False)
('37 ZeroPadding2D', False)
('38 DepthwiseConv2D', Fa

In [9]:
def lw(bottom_model, num_classes):
    """creates the top or head of the model that will be 
    placed ontop of the bottom layers"""

    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(512,activation='relu')(top_model)
    top_model = Dense(num_classes,activation='softmax')(top_model)
    return top_model

In [26]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

# Set our class number to 3 (As we have 3 classes to predict from)
num_classes = 1

FC_Head = lw(MobileNet, num_classes)

model = Model(inputs = MobileNet.input, outputs = FC_Head)

print(model.summary())

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128 

In [27]:
from keras.preprocessing.image import ImageDataGenerator
import os

base_path = os.getcwd()
print base_path
train_data_dir = '%s/yogesh/train/'%base_path
validation_data_dir = '%s/yogesh/test/'%base_path
print train_data_dir

# Let's use some data augmentaiton 
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=45,
      width_shift_range=0.3,
      height_shift_range=0.3,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# set our batch size (typically on most mid tier systems we'll use 16-32)
batch_size = 32
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical')

/home/bmkumar/Desktop/mlops/task3
/home/bmkumar/Desktop/mlops/task3/yogesh/train/
Found 399 images belonging to 1 classes.
Found 201 images belonging to 1 classes.


In [29]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import load_model

                     
checkpoint = ModelCheckpoint("facer.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]

# We use a very small learning rate 
model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])

# Enter the number of training and validation samples here
nb_train_samples = 400
nb_validation_samples = 200

# We only train 5 EPOCHS 
epochs = 10
batch_size = 64

history = model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

classifier = load_model('facer.h5')

Epoch 1/10


InvalidArgumentError:  Received a label value of 1 which is outside the valid range of [0, 1).  Label values: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
	 [[node loss_6/dense_16_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits (defined at /home/bmkumar/.local/lib/python2.7/site-packages/keras/backend/tensorflow_backend.py:3009) ]] [Op:__inference_keras_scratch_graph_19987]

Function call stack:
keras_scratch_graph
